In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib import pyplot
import os
import cv2
import random
import concurrent.futures
import time
import sklearn
print(tf.__version__)

In [ ]:

import tensorflow as tf
BATCH_SIZE = 1024
SHUFFLE_BUFFER = 2000
NUM_CLASSES = 100
IMAGE_SIZE = 224
def _parse_function(proto):
    # define your tfrecord again. Remember that you saved your image as a string.
    keys_to_features = {'image': tf.io.FixedLenFeature([], tf.string),
                        "class": tf.io.FixedLenFeature([], tf.int64)}
    # Load one example
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    # Turn your saved image string into an array
    image = decode_image(parsed_features['image'])
    label = tf.cast(parsed_features['class'], tf.int32)
    return image, label
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [IMAGE_SIZE, IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image
def create_dataset(filepath):    
    # This works with arrays as well
    dataset = tf.data.TFRecordDataset(filepath)
    # Maps the parser on every filepath in the array. You can set the number of parallel loaders here
    dataset = dataset.map(_parse_function, num_parallel_calls=4)
    # Set the number of datapoints you want to load and shuffle 
    dataset = dataset.shuffle(SHUFFLE_BUFFER)
    # Set the batchsize
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
    # Create an iterator
    iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)

    # Create your tf representation of the iterator
    image, label = iterator.get_next()
    # Bring your picture back in shape
    # Create a one hot array for your labels
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

In [ ]:

import tensorflow as tf
from tensorflow.python import keras as keras
from tensorflow.keras.applications import MobileNetV2

PATH = '../input/tpu-getting-started/tfrecords-jpeg-224x224/train'
fn = os.listdir(PATH)
filepath = [os.path.join(PATH, ele) for ele in fn]
#Get your datatensors
train_image, train_label = create_dataset(filepath)

PATH = '../input/tpu-getting-started/tfrecords-jpeg-224x224/val'
fn = os.listdir(PATH)
filepath = [os.path.join(PATH, ele) for ele in fn]
#Get your datatensors
val_image, val_label = create_dataset(filepath)


In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
SEED = 100
random_rotation = tf.keras.layers.experimental.preprocessing.RandomRotation(3.142/2, seed=SEED)
random_flip = tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical", seed=SEED)
random_zoom = tf.keras.layers.experimental.preprocessing.RandomZoom((0, 0.25), seed=SEED)
random_translate = tf.keras.layers.experimental.preprocessing.RandomTranslation((-0, 0.25), (-0, 0.25), seed=SEED)
with tf.device('/cpu:0'):
    train_dataset = tf.data.Dataset.from_tensor_slices((train_image, train_label)).batch(BATCH_SIZE)
    train_dataset = train_dataset.prefetch(buffer_size = tf.data.AUTOTUNE).shuffle(100)
    del train_image, train_label
    val_dataset = tf.data.Dataset.from_tensor_slices((val_image, val_label)).batch(32)
    val_dataset = val_dataset.prefetch(buffer_size = tf.data.AUTOTUNE).shuffle(100)
    del val_image, val_label
def normalize(imgs, label):
    imgs = random_rotation.call(imgs)
    imgs = random_flip.call(imgs)
    imgs = random_zoom.call(imgs)
    imgs = random_translate.call(imgs)
    return tf.cast(imgs, tf.float16)/255, label
train_dataset = train_dataset.map(normalize, num_parallel_calls=4)
val_dataset = val_dataset.map(normalize, num_parallel_calls=4)

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
    base_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),\
                                                   weights='imagenet', pooling = 'max', alpha=1.3)
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Dense(100, activation='softmax')
    ])
    optimizer = tf.keras.optimizers.Adam(0.001)
    epoch_auc = tf.keras.metrics.AUC(num_thresholds=200)
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
train_loss_history = []
val_loss_history = []
dist_train_dataset = strategy.experimental_distribute_dataset(train_dataset)
dist_val_dataset = strategy.experimental_distribute_dataset(val_dataset)


In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import accuracy_score
def compute_loss(labels, predictions):
    per_example_loss = loss_object(labels, predictions)
    return tf.nn.compute_average_loss(per_example_loss, global_batch_size=BATCH_SIZE)
def compute_acc(labels, predictions):
    return accuracy_score(labels, predictions)
def train_step(inputs):
    images, labels = inputs
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        loss_value = compute_loss(labels, logits)
        epoch_auc.update_state(labels, logits)
        auc = epoch_auc.result()
    train_loss_history.append(loss_value)
    grads = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss_value, auc
@tf.function
def distributed_train_step(dist_inputs):
    per_replica_losses, per_replica_auc = strategy.run(train_step, args=(dist_inputs,))
    loss = strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)
    auc = strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_auc,
                         axis=None)
    return loss, auc
def val_step(inputs):
    images, labels = inputs
    with tf.GradientTape() as tape:
        logits = model(images, training=False)
        loss_value = compute_loss(labels, logits)
        epoch_auc.update_state(labels, logits)
        auc = epoch_auc.result()
    val_loss_history.append(loss_value)
    return loss_value, auc
@tf.function
def distributed_val_step(dist_inputs):
    per_replica_losses, per_replica_auc = strategy.run(val_step, args=(dist_inputs,))
    loss = strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)
    auc = strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_auc,
                         axis=None)
    return loss, auc
def train(epochs, verbose=1):
    for epoch in range(epochs):
        start = time.time()
        i = 0
        print ('\nEpoch {}/{} '.format(epoch+1, epochs))
        for data in dist_train_dataset:
            loss, auc = distributed_train_step(data)
            
            percent = float(i+1) * 100 / len(train_dataset)
            arrow   = '-' * int(percent/100 * 20 - 1) + '>'
            spaces  = ' ' * (20 - len(arrow))
            if(verbose):    
                print('\rTraining: [%s%s] %d %% - Training Loss: %f - Training AUC: %f'% (arrow, spaces, percent, loss, auc), end='', flush=True)
            i += 1
        if(not verbose):
            print(' Epoch Loss: ', loss.numpy())
        i = 0
        if(verbose):
            print(" -", int(time.time()-start), "s", end="")
            print()
        start = time.time()
        for data in dist_val_dataset:
            loss, auc = distributed_val_step(data)
            percent = float(i+1) * 100 / len(val_dataset)
            arrow   = '-' * int(percent/100 * 20 - 1) + '>'
            spaces  = ' ' * (20 - len(arrow))
            if(verbose):    
                print('\rValidate: [%s%s] %d %% - Validation Loss: %f - Validation AUC: %f'% (arrow, spaces, percent, loss, auc), end='', flush=True)
            i += 1
        if(verbose):
            print(" -", int(time.time()-start), "s")

In [ ]:
train(50, verbose=1)

In [ ]:
model.save('./mobilenetv2.h5')